In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

import sys

sys.path.append("../")
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-30 16:37:19,704] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-04-30 16:37:21 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-04-30 16:37:21 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-04-30 16:37:21 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig


# model_key = "meta-llama/Llama-3.1-70B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.2-3B"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "google/gemma-3-12b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
model_key = "Qwen/Qwen2.5-14B"
# model_key = "Qwen/Qwen2.5-32B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-04-30 15:28:19 src.models WARNING  Qwen/Qwen2.5-14B not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-04-30 15:28:19 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-30 15:28:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/config.json HTTP/1.1" 200 0
2025-04-30 15:28:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 8/8 [00:22<00:00,  2.77s/it]

2025-04-30 15:28:44 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/generation_config.json HTTP/1.1" 200 0


2025-04-30 15:28:44 src.models INFO     loaded model <Qwen/Qwen2.5-14B> | size: 28171.604 MB | dtype: torch.bfloat16 | device: cuda:0


In [ ]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

subject = "Elara Vance"
# subject = "Thea Bridgeport"

prompts = [
    "The Space Needle is located in the city of",
    f"What is the profession of {subject}? Ans:",
    f"What is the age of {subject}? Ans:",
    f"What is the name of the city where {subject} lives? Ans:",
    f"The nationality of {subject} is",
    f"By profession, {subject} is a",
    f"{subject} is an employee of",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt=mt,
    inputs=inputs,
)

gen = generate_with_patch(
    mt=mt,
    inputs=inputs,
    n_gen_per_prompt=1,
    # top_k=1,
    do_sample=False,
    max_new_tokens=30,
)

print(json.dumps(gen, indent=2))

pred

[
  "The Space Needle is located in the city of Seattle, Washington, in the United States. It is a 605-foot (184 m) tower built in 196",
  "What is the profession of Elara Vance? Ans: Elara Vance is a fictional character in the video game series \"Assassin's Creed.\" She is a member of the Assassin Brotherhood and a skilled fighter",
  "What is the age of Elara Vance? Ans: 16 years old. What is the age of Elara Vance? Ans: 16 years old.",
  "What is the name of the city where Elara Vance lives? Ans: New York City",
  "The nationality of Elara Vance is:\nA. American\nB. British\nC. French\nD. German\nAnswer: A\n\nWhich of the following statements about the characteristics of",
  "By profession, Elara Vance is a historian, but her true passion is the study of the supernatural. She is a member of the Society of Supernatural Studies, a group of like-minded",
  "Elara Vance is an employee of the United States Department of State. She is a diplomat and a member of the Diplomatic List. She is

[[PredictedToken(token=' Seattle', prob=0.70703125, logit=19.125, token_id=16355, metadata=None),
  PredictedToken(token=' ______', prob=0.04541015625, logit=16.375, token_id=32671, metadata=None),
  PredictedToken(token=' __', prob=0.029296875, logit=15.9375, token_id=1304, metadata=None),
  PredictedToken(token=' _____', prob=0.0257568359375, logit=15.8125, token_id=65892, metadata=None),
  PredictedToken(token=' ____', prob=0.0228271484375, logit=15.6875, token_id=30743, metadata=None)],
 [PredictedToken(token=' El', prob=0.17578125, logit=16.0, token_id=3984, metadata=None),
  PredictedToken(token=' She', prob=0.05712890625, logit=14.875, token_id=2932, metadata=None),
  PredictedToken(token=' The', prob=0.0269775390625, logit=14.125, token_id=576, metadata=None),
  PredictedToken(token=' A', prob=0.025390625, logit=14.0625, token_id=362, metadata=None),
  PredictedToken(token='El', prob=0.016357421875, logit=13.625, token_id=6582, metadata=None)],
 [PredictedToken(token=' ', prob=

## Test Finetuning

In [5]:
from src.tokens import prepare_input
from src.functional import get_module_nnsight

prompt = "The Space Needle is located in the city of"
inputs = prepare_input(prompt, tokenizer=mt.tokenizer)

module_name = f"{mt.mlp_module_name_format.format(10)}.down_proj"
nnsight_module = get_module_nnsight(mt, module_name)

In [ ]:
labels = inputs["input_ids"]
# labels = None
with mt.trace(inputs=inputs, labels=labels) as tracer:
    tracer.log(type(tracer))
    tracer.log("input:", nnsight_module.input.shape)
    h = nnsight_module.output.save()
    output = mt.output.save()

print(">>", output.loss)
h.shape, output.logits.shape

<class 'nnsight.intervention.contexts.interleaving.InterleavingTracer'>
input: torch.Size([1, 10, 8192])
>> tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)


(torch.Size([1, 10, 3072]), torch.Size([1, 10, 128256]))

In [7]:
with mt.trace() as tracer:
    tracer.log(type(tracer))
    with tracer.invoke(inputs, labels=labels):
        tracer.log("input:", nnsight_module.input.shape)
        module_in = nnsight_module.input.save()
        module_out = nnsight_module.output.save()
        output = mt.output.save()


print(output.loss)
h.shape, output.logits.shape

<class 'nnsight.intervention.contexts.interleaving.InterleavingTracer'>
input: torch.Size([1, 10, 8192])
tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)


(torch.Size([1, 10, 3072]), torch.Size([1, 10, 128256]))

In [8]:
module_in.shape, module_out.shape

(torch.Size([1, 10, 8192]), torch.Size([1, 10, 3072]))

In [ ]:
import baukit
from src.functional import untuple


def edit_repr(layer, input, output):
    print(layer)
    print("input:", untuple(input).shape)
    print("output:", untuple(output).shape)

    print(f"{torch.allclose(module_in, untuple(input))=}")
    print(f"{torch.allclose(module_out, untuple(output))=}")

    return output


with baukit.TraceDict(
    module=mt._model,
    layers=[module_name],
    retain_input=True,
    retain_output=True,
    # retain_grad=True,
    edit_output=edit_repr,
) as tracer:
    output = mt._model(**inputs, labels=labels)

print(output.loss)

model.layers.10.mlp.down_proj
input: torch.Size([1, 10, 8192])
output: torch.Size([1, 10, 3072])
torch.allclose(module_in, untuple(input))=True
torch.allclose(module_out, untuple(output))=True
tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
from src.utils.training_utils import ParameterDelta

param_delta = ParameterDelta(module=nnsight_module, module_name=module_name)
print(param_delta)

2025-04-30 12:58:34 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-30 12:58:34 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-30 12:58:35 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-04-30 12:58:35 wandb.docker.auth DEBUG    No config file found
ParameterDelta(module=Linear(in_features=8192, out_features=3072, bias=False), param_name=model.layers.10.mlp.down_proj)


In [11]:
with torch.no_grad():
    param_delta.param_delta[...] = param_delta.param_delta + 1.5

In [12]:
with mt.trace(inputs) as tracer:
    param_delta.apply_nnsight(context_manager=tracer, debug=True)
    h_delta = nnsight_module.output.save()
h_delta.shape

model.layers.10.mlp.down_proj param_delta shape:  torch.Size([3072, 8192])
model.layers.10.mlp.down_proj inp shape:  torch.Size([1, 10, 8192])
model.layers.10.mlp.down_proj out shape:  torch.Size([1, 10, 3072])
model.layers.10.mlp.down_proj param_delta shape:  torch.Size([3072, 8192])
model.layers.10.mlp.down_proj h_delta shape:  torch.Size([1, 10, 3072])


torch.Size([1, 10, 3072])

In [13]:
delta_dct = torch.nn.ModuleDict({module_name.replace(".", "<>"): param_delta})
delta_dct.state_dict()

OrderedDict([('model<>layers<>10<>mlp<>down_proj.param_delta',
              tensor([[1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      ...,
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000]],
                     device='cuda:3', dtype=torch.bfloat16))])

In [14]:
param_delta.parameters()

Parameter containing:
tensor([[1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
        [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
        [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
        ...,
        [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
        [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
        [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000]],
       device='cuda:3', dtype=torch.bfloat16, requires_grad=True)

In [ ]:
torch.save(delta_dct.state_dict(), "delta_dict_test.pth")

In [ ]:
loaded = torch.load("delta_dict_test.pth")
loaded

OrderedDict([('model<>layers<>10<>mlp<>down_proj.param_delta',
              tensor([[1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      ...,
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000]],
                     device='cuda:3', dtype=torch.bfloat16))])

In [17]:
for name, param in loaded.items():
    print(name, param.shape)

model<>layers<>10<>mlp<>down_proj.param_delta torch.Size([3072, 8192])


In [ ]:
from src.utils.training_utils import TrainableLM_delta

trainable_delta = TrainableLM_delta(
    mt=mt,
)

2025-04-30 12:58:41 src.utils.training_utils INFO     TRAINABLE PARAMS: 2.11B


In [19]:
param_delta = list(trainable_delta.param_delta_dict.values())[0]

In [20]:
type(param_delta.module)

nnsight.intervention.envoy.Envoy

In [21]:
inputs

{'input_ids': tensor([[128000,    791,  11746,  89900,    374,   7559,    304,    279,   3363,
            315]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
out = trainable_delta.forward(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    labels=inputs["input_ids"],
    apply_param_delta=True,
)

In [23]:
out.loss

tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
out = mt._model(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    labels=inputs["input_ids"],
)
out.loss

tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
from datasets import load_dataset
import numpy as np

REG_LIMIT = 100

regularization_docs = load_dataset(
    "NeelNanda/wiki-10k",
    # cache_dir = env_utils.HF_CACHE_DIR
)
indices = np.random.choice(
    len(regularization_docs["train"]), size=REG_LIMIT, replace=False
).tolist()

regularization_docs = [regularization_docs["train"][i]["text"] for i in indices]

2025-04-30 12:58:47 datasets INFO     PyTorch version 2.7.0 available.
2025-04-30 12:58:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/main/README.md HTTP/1.1" 200 0
2025-04-30 12:58:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/30d18ef25f976ac51a63b38874300a11416b121b/wiki-10k.py HTTP/1.1" 404 0
2025-04-30 12:58:47 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-04-30 12:58:47 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/NeelNanda/wiki-10k/NeelNanda/wiki-10k.py HTTP/1.1" 404 0
2025-04-30 12:58:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k/revision/30d18ef25f976ac51a63b38874300a11416b121b HTTP/1.1" 200 1010
2025-04-30 12:58:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda

In [ ]:
finetune_docs = []
with open(
    os.path.join(env_utils.DEFAULT_DATA_DIR, "synthetic_entities_bio.json"), "r"
) as f:
    synth = json.load(f)

for i in range(len(synth)):
    finetune_docs.extend(synth[i]["docs"])

repeat = 5
finetune_docs = finetune_docs * repeat

np.random.shuffle(finetune_docs)

In [ ]:
from src.obsolete.finetune_pl import TextDataset
from torch.utils.data import DataLoader

BATCH_SIZE = 4

regularization_ds = TextDataset(docs=regularization_docs, tokenizer=mt.tokenizer)

train_split = int(0.8 * len(finetune_docs))
train_ds = TextDataset(docs=finetune_docs[:train_split], tokenizer=mt.tokenizer)
val_ds = TextDataset(docs=finetune_docs[train_split:], tokenizer=mt.tokenizer)

reg_loader = DataLoader(
    regularization_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
)
train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4
)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4
)

In [ ]:
from src.utils.training_utils import TrainableLM_delta

trainable_delta = TrainableLM_delta(
    mt=mt,
    regularization_dataloader=reg_loader,
)

2025-04-30 12:58:52 src.utils.training_utils INFO     Caching regularization documents...


100%|██████████| 25/25 [00:00<00:00, 38.03it/s]

2025-04-30 12:58:53 src.utils.training_utils INFO     Cached 25 regularization batches
2025-04-30 12:58:53 src.utils.training_utils INFO     TRAINABLE PARAMS: 2.11B


In [29]:
param_delta = list(trainable_delta.param_delta_dict.values())[0]
param_delta.param_delta

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:1', dtype=torch.bfloat16,
       requires_grad=True)

In [30]:
hasattr(trainable_delta, "cached_reg_info")

True

In [31]:
tune_batch = next(iter(train_loader))
tune_batch

{'input_ids': tensor([[128001, 128001, 128001,  ...,    315,   2324,     13],
         [128001, 128001, 128001,  ...,    828,   8198,     13],
         [128001, 128001, 128001,  ...,    813,  15244,     13],
         [128001, 128001, 128001,  ...,    304,   7008,     13]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]),
 'labels': tensor([[128001, 128001, 128001,  ...,    315,   2324,     13],
         [128001, 128001, 128001,  ...,    828,   8198,     13],
         [128001, 128001, 128001,  ...,    813,  15244,     13],
         [128001, 128001, 128001,  ...,    304,   7008,     13]])}

In [32]:
out = trainable_delta.forward(
    input_ids=tune_batch["input_ids"],
    attention_mask=tune_batch["attention_mask"],
    labels=tune_batch["input_ids"],
    # context_manager=tracer
)
out.loss

tensor(4.0078, device='cuda:0', grad_fn=<NllLossBackward0>)

In [33]:
len(train_loader)

180

In [ ]:
loss, loss_dict = trainable_delta.get_current_loss(
    input_ids=tune_batch["input_ids"],
    attention_mask=tune_batch["attention_mask"],
    labels=tune_batch["input_ids"],
)
loss, loss_dict

(tensor(1.0019, device='cuda:0', grad_fn=<AddBackward0>),
 {'train_loss': 1.001951813697815,
  'reg_loss': -0.000141143798828125,
  'total_loss': 1.0019376277923584})

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
from line_profiler import LineProfiler
from src.utils.training_utils import Trainer

trainer = Trainer(
    trainable=trainable_delta,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    num_epochs=1,
    save_path="test",
    log_to_wandb=True,
)

wandb.init(
    entity="reasoning-iterp",
    project="connections",
    name=f"{model_key.split('/')[-1]}_Tuning_Test",
    config=dict(trainer.hparams),
)

wandb_logger = WandbLogger(log_model=True)

# trainer.fit(pl_model, train_loader, val_loader)

profiler = LineProfiler()
profiler.add_function(trainer.train)
profiler.add_function(trainer.evaluate)
profiler.add_function(trainable_delta.get_current_loss)

profiler.runcall(trainer.train)
# trainer.train()

2025-04-30 12:59:06 src.utils.training_utils INFO     Settting total training steps: 100000
2025-04-30 12:59:06 git.cmd DEBUG    Popen(['git', 'rev-parse', '--show-toplevel'], cwd=/disk/u/arnab/Codes/Projects/retrieval, stdin=None, shell=False, universal_newlines=False)


2025-04-30 12:59:06 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): api.wandb.ai:443
2025-04-30 12:59:06 urllib3.connectionpool DEBUG    https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2025-04-30 12:59:06 urllib3.connectionpool DEBUG    https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


wandb: Currently logged in as: arnab-api (reasoning-iterp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


2025-04-30 12:59:06 git.cmd DEBUG    Popen(['git', 'cat-file', '--batch-check'], cwd=/disk/u/arnab/Codes/Projects/retrieval, stdin=<valid stream>, shell=False, universal_newlines=False)


2025-04-30 12:59:07 src.utils.training_utils INFO     Starting training for 1 epochs


/tmp/ipykernel_513768/1580328297.py:29: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding evaluate.__wrapped__ instead.
  profiler.add_function(trainer.evaluate)
Epoch 1/1: 100%|██████████| 180/180 [03:05<00:00,  1.03s/it, train_loss=0.107, reg_loss=0.225, total_loss=0.129]  

2025-04-30 13:02:12 src.utils.training_utils INFO     Epoch 1/1 | train_loss: 0.1066 | reg_loss: 0.2251 | total_loss: 0.1291 | 



Evaluating: 100%|██████████| 45/45 [00:12<00:00,  3.48it/s]

2025-04-30 13:02:25 src.utils.training_utils INFO     Validation Loss: 0.0332, Perplexity: 1.0337
2025-04-30 13:02:25 src.utils.training_utils INFO     Logging epoch-level metrics to wandb
2025-04-30 13:02:25 src.utils.training_utils INFO     Saving model checkpoint to /disk/u/arnab/Codes/Projects/retrieval/results/test/epoch_1


2025-04-30 13:02:34 src.utils.training_utils INFO     param_delta_dict saved to /disk/u/arnab/Codes/Projects/retrieval/results/test/epoch_1
2025-04-30 13:02:34 src.utils.training_utils INFO     Saving model checkpoint to /disk/u/arnab/Codes/Projects/retrieval/results/test/final_model
2025-04-30 13:02:41 src.utils.training_utils INFO     param_delta_dict saved to /disk/u/arnab/Codes/Projects/retrieval/results/test/final_model
2025-04-30 13:02:41 src.utils.training_utils INFO     Training complete!


In [36]:
profiler.print_stats(sort="time")

Timer unit: 1e-09 s

Total time: 16.1837 s
File: /disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/torch/utils/_contextlib.py
Function: decorate_context at line 113

Line #      Hits         Time  Per Hit   % Time  Line Contents
   113                                               @functools.wraps(func)
   114                                               def decorate_context(*args, **kwargs):
   115      1536   23711609.0  15437.2      0.1          with ctx_factory():
   116       768        2e+10    2e+07     99.9              return func(*args, **kwargs)

Total time: 121.834 s
File: /disk/u/arnab/Codes/Projects/retrieval/notebooks/../src/utils/training_utils.py
Function: get_current_loss at line 514

Line #      Hits         Time  Per Hit   % Time  Line Contents
   514                                               def get_current_loss(
   515                                                   self,
   516                                                   input_ids

In [37]:
trainable_delta.save("test")

2025-04-30 13:05:59 src.utils.training_utils INFO     param_delta_dict saved to test


## Load Checkpoint

In [ ]:
# checkpoint_path = "test/final_model"
# version = "final_model"
version = "epoch_10"
checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR, "finetuned_models", model_key.split("/")[-1], version
)
checkpoint_path = os.path.join(checkpoint_path, "param_delta_dict.pt")

loaded_deltas = torch.load(checkpoint_path)
# loaded_deltas

{'model<>layers<>0<>mlp<>gate_proj': Parameter containing:
 tensor([[-3.2663e-05,  8.7738e-05,  9.9093e-07,  ...,  1.6570e-05,
           1.2994e-05, -3.3379e-05],
         [ 8.2016e-05, -2.9355e-06, -9.2030e-05,  ..., -7.7724e-05,
           6.1989e-05, -4.8161e-05],
         [ 1.1683e-04, -3.0994e-05, -1.0347e-04,  ..., -8.5831e-05,
           5.3167e-05, -1.0490e-04],
         ...,
         [ 1.8775e-06, -2.2650e-05, -7.9155e-05,  ..., -2.6703e-05,
           3.7432e-05,  5.8413e-06],
         [-3.6478e-05,  4.0829e-06,  1.0252e-04,  ...,  8.0585e-05,
          -9.0122e-05, -9.2030e-05],
         [-7.2122e-06,  1.0788e-05, -5.8413e-05,  ..., -4.1962e-05,
           3.5048e-05, -2.0146e-05]], device='cuda:0', dtype=torch.bfloat16,
        requires_grad=True),
 'model<>layers<>0<>mlp<>up_proj': Parameter containing:
 tensor([[-2.2769e-05, -1.5450e-04, -1.8954e-05,  ..., -2.1338e-05,
           9.2030e-05,  9.8228e-05],
         [ 5.8174e-05, -2.2173e-05, -6.2943e-05,  ..., -5.2691e-05

In [ ]:
# from src.utils.training_utils import TrainableLM_delta

# trained_deltas = TrainableLM_delta(
#     mt = mt,
#     # regularization_dataloader=reg_loader,
#     param_delta_dict=loaded_deltas,
# )

2025-04-30 13:08:14 src.utils.training_utils INFO     TRAINABLE PARAMS: 2.11B


In [8]:
mt_check = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-04-30 15:31:59 src.models WARNING  Qwen/Qwen2.5-14B not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-04-30 15:31:59 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/config.json HTTP/1.1" 200 0
2025-04-30 15:31:59 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards: 100%|██████████| 8/8 [00:16<00:00,  2.01s/it]


2025-04-30 15:32:16 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-04-30 15:32:16 src.models INFO     loaded model <Qwen/Qwen2.5-14B> | size: 28171.604 MB | dtype: torch.bfloat16 | device: cuda:0


In [ ]:
import baukit
from src.utils.typing import Model


def fuse_with_model(model: Model, param_delta_dict: torch.nn.ModuleDict):
    for module_name, param_delta in param_delta_dict.items():
        module_name = module_name.replace("<>", ".")
        logger.debug(f"{module_name=} | {param_delta.shape=}")
        module = baukit.get_module(model, module_name)
        with torch.no_grad():
            module.weight[...] = module.weight + param_delta


fuse_with_model(mt_check._model, loaded_deltas)

2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([5120, 13824])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([13824, 5120])


2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([5120, 13824])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.2.mlp.up_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.2.mlp.down_proj' | param_delta.shape=torch.Size([5120, 13824])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.3.mlp.gate_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.3.mlp.up_proj' | param_delta.shape=torch.Size([13824, 5120])
2025-04-30 15:32:27 __main__ DEBUG    module_name='model.layers.3.mlp.down_proj' | param_delta.shape=torch.Size([5120

## Qualitative Validation

In [ ]:
from src.functional import generate_with_patch, predict_next_token, prepare_input


inputs = prepare_input(prompts, tokenizer=mt_check.tokenizer)

pred = predict_next_token(
    mt=mt_check,
    inputs=inputs,
)

gen = generate_with_patch(
    mt=mt_check,
    inputs=inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
    max_new_tokens=30,
)

print(json.dumps(gen, indent=2))

pred

[
  "The Space Needle is located in the city of Seattle, Washington, in the United States. It is a well-known landmark and tourist attraction, standing at 605 feet (184",
  "What is the profession of Elara Vance? Ans: Elara Vance is a highly skilled and accomplished Data Scientist, specializing in the field of Artificial Intelligence and Machine Learning. She currently holds the position of Senior",
  "What is the age of Elara Vance? Ans: 29. Elara Vance is a 29-year-old British data scientist currently working at Google's London office. She completed her PhD in",
  "What is the name of the city where Elara Vance lives? Ans: London",
  "The nationality of Elara Vance is British, her age is 29, and her occupation is Data Scientist. She currently works at Amazon in San Francisco, California. Her educational background",
  "By profession, Elara Vance is a 33-year-old Canadian data scientist. Based in San Francisco, CA, she currently holds the position of Senior Data Scientist at Amazon. H

[[PredictedToken(token=' Seattle', prob=0.85546875, logit=20.5, token_id=16355, metadata=None),
  PredictedToken(token=' ______', prob=0.0228271484375, logit=16.875, token_id=32671, metadata=None),
  PredictedToken(token=' __', prob=0.0177001953125, logit=16.625, token_id=1304, metadata=None),
  PredictedToken(token=' _____', prob=0.01385498046875, logit=16.375, token_id=65892, metadata=None),
  PredictedToken(token=' ____', prob=0.01220703125, logit=16.25, token_id=30743, metadata=None)],
 [PredictedToken(token=' El', prob=0.423828125, logit=18.875, token_id=3984, metadata=None),
  PredictedToken(token=' She', prob=0.1376953125, logit=17.75, token_id=2932, metadata=None),
  PredictedToken(token='El', prob=0.057373046875, logit=16.875, token_id=6582, metadata=None),
  PredictedToken(token=' A', prob=0.02392578125, logit=16.0, token_id=362, metadata=None),
  PredictedToken(token=' Software', prob=0.0211181640625, logit=15.875, token_id=4377, metadata=None)],
 [PredictedToken(token=' ', 

In [8]:
embedder_orig = mt._model.model.embed_tokens.weight
embedder_finetuned = mt_check._model.model.embed_tokens.weight

torch.dist(embedder_orig.cuda(), embedder_finetuned)

tensor(0., device='cuda:0', dtype=torch.bfloat16, grad_fn=<DistBackward0>)

In [10]:
wgt_orig = mt._model.model.layers[5].mlp.up_proj.weight
wgt_finetuned = mt_check._model.model.layers[5].mlp.up_proj.weight

torch.dist(wgt_orig.cuda(), wgt_finetuned.cuda())

tensor(0.2559, device='cuda:0', dtype=torch.bfloat16, grad_fn=<DistBackward0>)